An X-ray C-arm can be modeled as a pinhole camera with its own extrinsic and intrinsic matrices. 
This module provides utilities for parsing these matrices and working with rigid transforms.

In [ ]:
#| default_exp calibration

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch

## Computing a perspective projection

Given an `extrinsic` and `intrinsic` camera matrix, we can compute the perspective projection of a batch of points.
This is used for computing where fiducials in world coordinates get mapped onto the image plane.

In [ ]:
#| export
from typing import Optional

from beartype import beartype
from diffdrr.pose import RigidTransform
from jaxtyping import Float, jaxtyped

In [ ]:
#| export
@jaxtyped(typechecker=beartype)
def perspective_projection(
    extrinsic: RigidTransform,  # Extrinsic camera matrix (world to camera)
    intrinsic: Float[torch.Tensor, "3 3"],  # Intrinsic camera matrix (camera to image)
    x: Float[torch.Tensor, "b n 3"],  # World coordinates
) -> Float[torch.Tensor, "b n 2"]:
    x = extrinsic(x)
    x = torch.einsum("ij, bnj -> bni", intrinsic, x)
    z = x[..., -1].unsqueeze(-1).clone()
    x = x / z
    return x[..., :2]

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()